In [ ]:
# ==========================================
# CELL 1: INSTALLATION & SETUP
# ==========================================
!pip install -qU langchain==0.1.0 langchain-google-genai==0.0.9 langchain-community==0.0.19 python-dotenv

In [ ]:
pip install google-search-results

In [ ]:
from langchain_community.utilities import SerpAPIWrapper

In [ ]:
import os
from google.colab import userdata
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_google_genai import ChatGoogleGenerativeAI

# 🔑 SETUP API KEY
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1,convert_system_message_to_human=True)

os.environ["SERPAPI_API_KEY"] = "YOUR_API_KEY"

print("✅ Libraries installed and LLM ready.")

✅ Libraries installed and LLM ready.


In [ ]:
# Define the tools [workers]
def news_search(query):

    """Worker 1: The News Reporter.
    Strictly fails if asked for numbers/prices."""

    # --- LOGIC: The "Self-Realization" of Failure ---
    # If the query is about math/money, this tool intentionally crashes
    # to trigger the hand-off.
    forbidden_keywords = ["price", "trading", "ratio", "value", "number"]
    if any(word in query.lower() for word in forbidden_keywords):
        raise ValueError("I am a News Reporter! I cannot handle specific numbers or stock prices.")

    # Otherwise, perform a news search
    search = SerpAPIWrapper(params={"tbm": "nws"}) # tbm=nws forces Google News results
    return search.run(f"{query} latest headlines")

def stock_search(query):
    """Worker 2: The Stock Broker.
    Finds specific market data."""
    #search = SerpAPIWrapper(params={"engine": "google_finance"})
    # Or standard search if finance engine not available in your plan:
    search = SerpAPIWrapper(params={"engine": "google"})
    #return search.run(f"{query} stock price") --->This code is also running

    #Following logic is to solve Identity Crisis problem -
    #This forces the LLM to trust the price even if the name looks different.
    #(e.g., Google $\to$ Alphabet, Facebook $\to$ Meta, Zomato $\to$ Eternal).

    result = search.run(f"{query} stock price")

    # <--- THE UNIVERSAL FIX --->
    # We append a GENERIC instruction to the result.
    # This tells the Agent: "If you see a price, trust it, even if the name is weird."
    return (
        f"Search Result: {result}\n"
        f"SYSTEM NOTE: The company name in the search result might differ from the query "
        f"(e.g., Parent Company Name or Legal Entity Name). "
        f"If a stock price is visible in the result, assume it is the correct data for '{query}' and return it."
    )


# Step 2 : Wrap them as langchain tools!
news_tool = Tool(
    name="NewsSearch",
    func=news_search,
    description="Useful ONLY for finding text articles and headlines. Fails on numbers/prices."
)

stock_tool = Tool(
    name="StockSearch",
    func=stock_search,
    description="Useful for finding current stock prices, market ratios, and numeric data."
)

In [ ]:
# ==========================================
# DECENTRALIZED ARCHITECTURE (The Hand-off)
# ==========================================
from langchain.agents import initialize_agent, AgentType, Tool

# --- Step 1: Create Individual Agents ---
# We use ZERO_SHOT_REACT_DESCRIPTION because it handles simple string inputs better.

reporter_agent = initialize_agent(
    tools=[news_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # <--- CHANGED THIS
    verbose=True,
    handle_parsing_errors=True # Helps agent recover if it messes up formatting
)

stock_broker_agent = initialize_agent(
    tools=[stock_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # <--- CHANGED THIS
    verbose=True,
    handle_parsing_errors=True
)

# --- Step 2: The Hand-off Logic (The "Glue") ---
def run_decentralized_flow(query):
    print(f"\n🔹 User Query: '{query}'")
    print("👉 Attempting: [News Reporter Agent]...")

    try:
        # 1. Always try the News Reporter first
        # We wrap this in a try/except because the tool is designed to crash on price queries
        response = reporter_agent.run(query)

        return response

    except ValueError as e:
        # 2. The Fallback (Hand-off)
        print(f"\n❌ News Agent FAILED: {e}")
        print("🔄 HAND-OFF TRIGGERED: Calling [Stock Broker Agent] to save the day...\n")

        # 3. Call the Specialist
        return stock_broker_agent.run(query)

    except Exception as e:
        # Catch generic LangChain errors
        print(f"\n❌ General Error in Reporter: {e}")
        print("🔄 HAND-OFF TRIGGERED: Calling [Stock Broker Agent]...")
        return stock_broker_agent.run(query)

# --- Step 3: Run It ---
print("\n🤖 DECENTRALIZED SYSTEM: Passing the baton.")
# Scenario: asking for a price (should trigger hand-off)
print("--------------------------------------------------")
final_answer = run_decentralized_flow("What is Zomato trading at?")
print(f"📢 Final Answer: {final_answer}")
print("--------------------------------------------------")


🤖 DECENTRALIZED SYSTEM: Passing the baton.
--------------------------------------------------

🔹 User Query: 'What is Zomato trading at?'
👉 Attempting: [News Reporter Agent]...


> Entering new AgentExecutor chain...
Action: NewsSearch
Action Input: Zomato stock price
❌ News Agent FAILED: I am a News Reporter! I cannot handle specific numbers or stock prices.
🔄 HAND-OFF TRIGGERED: Calling [Stock Broker Agent] to save the day...



> Entering new AgentExecutor chain...
Action: StockSearch
Action Input: Zomato
Observation: Search Result: {'type': 'finance_results', 'title': 'News', 'exchange': 'NSE', 'stock': 'ETERNAL', 'currency': 'INR', 'price': 284.5}
SYSTEM NOTE: The company name in the search result might differ from the query (e.g., Parent Company Name or Legal Entity Name). If a stock price is visible in the result, assume it is the correct data for 'Zomato' and return it.
Thought:I now know the final answer
Final Answer: Zomato is trading at 284.5 INR.

> Finished chain.
📢 Final